In [4]:
import pandas as pd
import ast
import numpy as np


movies = pd.read_csv('./movies_metadata.csv')
keywords = pd.read_csv('./credits.csv')
links = pd.read_csv('./links.csv')
ratings = pd.read_csv('./keywords.csv')
links = pd.read_csv('./links.csv')






C:\Users\juanj\AppData\Local\Temp\ipykernel_6932\3177823400.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('C:/Users/juanj/Documents/3ercuatri/modelado/modelado_p1/movies_metadata.csv')


In [6]:
import pandas as pd
def preprocesar(df):
    df.drop
    df.replace('', "NaN")
    

    

pd.to_numeric(movies['budget'],errors='ignore',downcast='integer')
for element in (movies,credits,keywords,ratings,links):
    preprocesar(element)
rows_a_eliminar=[44551,44552,44553]
movies=movies.drop(rows_a_eliminar)   #EL ID ES UNA FECHA

In [7]:
import ast
def get_lista_diccionarios_json (filename, atribute_name,row) -> [tuple]:  #estamos dentro de una columna json, en una fila (no iteramos)  #JURARIA QUE TODAS LAS TUMPLAS SON DE DOS ELEMENTOS
    dataframe = filename  #CROE QUE SE PUEDE QUITAR , EL FILENAME ES UNA VARIABLE Y YA HEMOS LEIDO ANTES 
    lista_diccionarios = ast.literal_eval(filename[str(atribute_name)].iloc[row])
    dict_primary_keys={'genres':'id',
                       'production_countries':'iso_3166_1',
                       'production_companies':'id',
                       'spoken_languages':'iso_639_1',
                       'keywords':'id'}
    lista=[]
    
    for diccionario in lista_diccionarios:
        name=diccionario['name']               #Animation,Comedy,English..
        id_atributo=diccionario[dict_primary_keys[atribute_name]]
        lista.append((id_atributo,name))
    
    return lista  



In [8]:
d={  #nombre entidades a crear: lista con nombre de sus atributos o columnas
    'movie_genres':['movie_id','genre_id'],
    'genres':['genre_id','name'],
    'movie_production_companies':['movie_id','company_id'],
    'production_companies':['company_id','name'],
    'movie_production_countries':['movie_id','country_code'],
    'countries':['iso_3166_1','name'],
    'movie_spoken_languages':['movie_id','language_code'],
    'languages':['iso_639_1 ','name'],
    'movie_keywords':['movie_id','keyword_id'],
    'keywords':['keyword_id','name']}

dict_keys=d.keys()
def get_csv_from_jsons_columns(filename, name, atribute_name,cols,d=d):  #unificar columnas,genera un csv,llama en un bucle muchas veces a get_dict_list
    
    lista_rows=[]
    c1=cols[0]
    c2=cols[1]
    for i in range(len(filename[atribute_name])):
        try:
            lista_de_tuplas=get_lista_diccionarios_json(filename,atribute_name,i)  
        except:
            continue
        
        for tupla in lista_de_tuplas:                                           
            
            lista_rows.append({c1:tupla[0],c2:tupla[1]})
        

    df = pd.DataFrame(lista_rows, columns=[c1,c2]) 
    df=df.drop_duplicates()
    df.to_csv(name+'.csv',index=False,header=True,encoding='utf-8') 
    
    
for tuplo in [(movies,'genres','genres',d['genres']),(movies,'countries','production_countries',d['countries']),(movies,'production_companies','production_companies',d['production_companies']),(movies,'languages','spoken_languages',d['languages']),(keywords,'keywords','keywords',d['keywords'])]:
    get_csv_from_jsons_columns(tuplo[0],tuplo[1],tuplo[2],tuplo[3])

In [9]:
def relation_with_movie_id_to_csv(filename,name,atribute_name,cols,d=d):  #DICCIONARIO LIST NEW COLUMNS
    lista_rows=[]
    c1=cols[0]
    c2=cols[1]
    for i in range(len(filename[atribute_name])):
        try:
            lista_de_tuplas=get_lista_diccionarios_json(filename,atribute_name,i)
        except:
            continue
        movie_id=filename['id'].iloc[i]
        
        for tupla in lista_de_tuplas:
            
            lista_rows.append({c1:movie_id,c2:tupla[0]})
            
    df = pd.DataFrame(lista_rows, columns=[c1,c2]) 
    df=df.drop_duplicates()
    df.to_csv(name+'.csv',index=False,header=True,encoding='utf-8') #por que con index false sigue saliendo??
    
for tuplo in [(movies,'movie_genres','genres',d['movie_genres']),(movies,'movie_production_countries','production_countries',d['movie_production_countries']),(movies,'movie_production_companies','production_companies',d['movie_production_companies']),(movies,'movie_spoken_languages','spoken_languages',d['movie_spoken_languages']),(keywords,'movie_keywords','keywords',d['movie_keywords'])]:
    relation_with_movie_id_to_csv(tuplo[0],tuplo[1],tuplo[2],tuplo[3])
    

In [12]:

#GENERAL:


def informacion_general(filename, name, atribute_list)-> list:
    """filename es el nombre del archivo del que sacamos los datos. Name es el nombre del csv que queremos, list es
        la lista de atributos que queremos sacar"""
    df = pd.read_csv(filename,low_memory=False, usecols= atribute_list)
    df_final = df.copy()
    
    df_final.to_csv(name+'.csv',index=False,encoding='utf-8')
informacion_general('./movies_metadata.csv' , "movie_basics", ['id', 'title', 'original_title', 'release_date', 'belongs_to_collection', 'original_language', 'adult', 'status'])
informacion_general('./movies_metadata.csv' , "movie_additional", ['id', 'imdb_id', 'homepage', 'budget', 'overview', 'popularity', 'vote_average', 'vote_count', 'poster_path', 'tagline', 'video', 'revenue', 'runtime'])
##COLLECTIONS: ES REDUNDANTE PERO NO SE COMO ARREGLARLO
def collections():
    lista_diccionarios = []
    for i in range(len(movies["belongs_to_collection"])):
        try:
            j = ast.literal_eval(movies["belongs_to_collection"].iloc[i])
            if type(j) != float:
                lista_diccionarios.append(j)
        except:
            pass
    print(lista_diccionarios)
    genres_df = pd.DataFrame(lista_diccionarios).drop_duplicates()
    genres_df.to_csv('collections.csv', index=False)

#PEOPLE-CREW-CAST
def elements(filename, name,column, atr):
    lista = []
    for i in range(len(filename[column])):
        j = ast.literal_eval(filename[column].iloc[i])
        for element in j:
            res = {key: element[key] for key in element.keys() & atr}
            lista.append(res)
    final_df = pd.DataFrame(lista).drop_duplicates()
    final_df.to_csv(name+'.csv', index=False)
    return final_df 
#####
def relations(filename, name,column, atr):
    lista = []
    for i in range(len(filename[column])):
        j = ast.literal_eval(filename[column].iloc[i])
        for element in j:
            res = {key: element[key] for key in element.keys() & atr}
            res['movie_id'] = filename['id'].iloc[i]
            lista.append(res)
    final_df = pd.DataFrame(lista).drop_duplicates()
    final_df.to_csv(name+'.csv', index=False)
    return final_df
relations(credits, 'movie_cast', 'cast', {'credit_id','id', 'character', 'cast_id', 'order'})
relations(credits, 'movie_crew', 'crew', {'credit_id','id', 'department', 'job', 'profile_path'})
###
def people_y_genders():
    cast = elements(credits, 'cast', 'cast', {'id', 'name', 'gender', 'profile_path'})
    crew = elements(credits, 'crew', 'crew', {'id', 'name', 'gender', 'profile_path'})
    merged_df = pd.merge(cast, crew)
    merged_df.to_csv('people.csv', index=False)
    genderdf = merged_df[["id", "gender"]].copy()
    genderdf =genderdf.drop_duplicates()
    genderdf.to_csv('gender.csv', index= False)
people_y_genders()

In [13]:
collections()

[{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}, {'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}, {'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}, {'id': 645, 'name': 'James Bond Collection', 'poster_path': '/HORpg5CSkmeQlAolx3bKMrKgfi.jpg', 'backdrop_path': '/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg'}, {'id': 117693, 'name': 'Balto Collection', 'poster_path': '/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg', 'backdrop_path': '/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg'}, {'id': 3167, 'name': 'Ace Ventura Collection', 'poster_path': '/qCxH543pScFed1CycwJ1nVgrkOc.jpg', 'backdrop_path': '/bswWgdDsLu0fhWMYUzLF8XgiK4h.jpg'}, {'id': 91698, 'name': 'Chili Palmer Collection', 'poster_path': '/ae3smJDdWrMJ77t